# Social Security cap and `expanded_income`

Why does raising the Social Security cap change `expanded_income`?

## Setup

In [1]:
import taxcalc as tc
import pandas as pd

## Create Calculators

In [2]:
recs = tc.Records.cps_constructor()

In [3]:
REMOVE_SS_CAP_REFORM = {'SS_Earnings_c': {2019: 1e+16}}

In [4]:
base = tc.Calculator(tc.Policy(), recs)
base.advance_to_year(2019)
base.calc_all()

In [5]:
pol = tc.Policy()
pol.implement_reform(REMOVE_SS_CAP_REFORM)
reform = tc.Calculator(pol, recs)
reform.advance_to_year(2019)
reform.calc_all()

## Compare
As expected, `aftertax_income` falls, by $89 billion.

In [6]:
def chg(metric):
    return reform.weighted_total(metric) - base.weighted_total(metric)

In [7]:
METRICS = ['aftertax_income', 'expanded_income', 'combined', 'iitax',
           'payrolltax', 'ptax_was']

In [8]:
chgs_b = pd.Series([chg(i) for i in METRICS], index=METRICS) / 1e9
chgs_b.astype(int)

aftertax_income    -88
expanded_income     63
combined           152
iitax               -5
payrolltax         157
ptax_was           127
dtype: int64

The rise in expanded income is half the change in payroll taxes from wages and salaries.

In [9]:
chg('expanded_income') / chg('ptax_was')

0.4999999999999914

These numbers match [this Comp run](https://www.compmodels.org/PSLmodels/Tax-Brain/41147/).

Using Comp with the PUF (https://www.compmodels.org/PSLmodels/Tax-Brain/41139/), the payroll tax liability rises by $191 billion, and there's virtually no change to income tax liability.

## Match tax units

Calculate each tax unit's difference in after-tax income, *assuming they bear the cost of the employer payroll tax hike.*

In [10]:
base_df = base.dataframe(['combined', 's006'])
reform_df = reform.dataframe(['combined'])

In [11]:
joined_df = base_df.join(reform_df, lsuffix='_base', rsuffix='_reform')
joined_df['combined_diff'] = (joined_df.combined_reform - 
                              joined_df.combined_base)
joined_df['combined_diff_m'] = joined_df.combined_diff * joined_df.s006 / 1e6
joined_df.combined_diff_m.sum()

152359.0333187517

In [12]:
joined_df[joined_df.combined_diff > 0].sample(10)

,combined_base,s006,combined_reform,combined_diff,combined_diff_m
291260,54501.082068,10.83,67330.711321,12829.629253,0.138945
304801,70187.407626,11.11,81433.594645,11246.187019,0.124945
447151,86595.135909,75.83,108615.217219,22020.081311,1.669783
120261,87075.600218,77.90,103900.202178,16824.601960,1.310636
96029,42878.331796,588.61,46575.892895,3697.561099,2.176421
285475,75593.209578,45.18,89021.421276,13428.211699,0.606687
37968,133214.300293,36.84,170879.351125,37665.050832,1.387580
80290,156954.557937,45.07,201479.570009,44525.012072,2.006742
412486,42384.685534,9.19,45691.967139,3307.281605,0.030394
217305,48903.652960,416.07,53388.512152,4484.859193,1.866015
